In [ ]:
from datasets import load_dataset
import numpy as np
from transformers import AutoTokenizer
import evaluate
from datasets import load_dataset, concatenate_datasets
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import json
import pandas as pd

# test_data = # TODO: Fill in here

# TODO: Please make sure the test data is in the format of:
# text: "This is a sample text."
# source: "human" or "ai"

# as a reference, here is how we transformed one of the evaluation datasets into the required format
# arxiv_chatGPT = load_dataset('json', data_files='arxiv_chatGPT.jsonl')

# arxiv_chatGPT_human = arxiv_chatGPT.map(lambda x: {'text': x['human_text'], 'source': 'human'}).select_columns(['text', 'source'])
# arxiv_chatGPT_machine = arxiv_chatGPT.map(lambda x: {'text': x['machine_text'], 'source': 'ai'}).select_columns(['text', 'source'])

# arxiv_chatGPT = concatenate_datasets([arxiv_chatGPT_human['train'], arxiv_chatGPT_machine['train']])
# arxiv_chatGPT = arxiv_chatGPT.shuffle()

In [2]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [3]:
# def preprocess_function(examples):
#     model_inputs = tokenizer(
#         examples['text'],
#         # padding='max_length',
#         truncation=True,
#         # max_length=1024,
#         # return_tensors='pt'
#     )
#     model_inputs["labels"] = [0 if s == 'human' else 1 for s in examples['source']]
    
#     return model_inputs

# tokenized_test_data = test_data.map(preprocess_function, batched=True)


In [4]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [5]:
import evaluate

accuracy = evaluate.load("accuracy")

In [6]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [7]:
id2label = {0: "Not AI Generated", 1: "AI Generated"}
label2id = {"Not AI Generated": 0, "AI Generated": 1}

In [ ]:
# initialize model
from transformers import AutoModelForSequenceClassification
import torch
def predict_single_text(text, model, tokenizer, device, max_len=512, threshold=0.5):
    encoded = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs["logits"]
        # softmax the logits
        logits = torch.softmax(logits, dim=1)
        # get probability of being AI generated
        probability = logits[0][1].item()

    label = 1 if probability >= threshold else 0
    return probability, label

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# --- Run prediction ---
tokenizer = AutoTokenizer.from_pretrained("ai_detection_model_final") # Replace ai_detection_model_final with your model name
tuned_model = AutoModelForSequenceClassification.from_pretrained(
    "ai_detection_model_final", num_labels=2, id2label=id2label, label2id=label2id
)
tuned_model.to(device)


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
# test set evaluation
correct = 0
tp = 0
fp = 0
tn = 0
fn = 0
total = 0
y_true = []
y_scores = []
for example in test_data:
    text = example['text']
    probability, predicted_label = predict_single_text(text, tuned_model, tokenizer, device)
    # print(f"Probability of being AI generated: {probability:.4f}")
    # print(f"Predicted label: {'AI Generated' if predicted_label == 1 else 'Not AI Generated'}")
    
    y_scores.append(probability)       
    if(predicted_label == 1 and example['source'] == 'ai') or (predicted_label == 0 and example['source'] == 'human'):
        correct += 1
    total += 1

    # get tp/fp/fn/tn
    if predicted_label == 1 and example['source'] == 'ai':
        tp += 1
        y_true.append(1)
    elif predicted_label == 1 and example['source'] == 'human':
        fp += 1
        y_true.append(0)
    elif predicted_label == 0 and example['source'] == 'ai':
        fn += 1
        y_true.append(1)
    elif predicted_label == 0 and example['source'] == 'human':
        tn += 1
        y_true.append(0)


print(f"Accuracy: {correct/total:.4f}")
print(f"Recall: {tp/(tp+fn):.4f}")
print(f"Precision: {tp/(tp+fp):.4f}")
print(f"F1: {2*tp/(2*tp+fp+fn):.4f}")

Accuracy: 0.9400
Recall: 0.9545
Precision: 0.9130
F1: 0.9333
